In [1]:
import os
from pathlib import Path
from matplotlib import pyplot as plt


In [2]:
from model_analysis import * 

# Complete legacy runs with latest files

To ensure consistency of processing legacy runs which may not have all the files for later processing,
are enhanced with those files filled in to the best of our abilities. 
This notebook defines those defaults.

In [3]:
run_manifest = read_manifest('run-manifest.csv')
run_manifest.tail()

,folder,version,last available data,number of countries,number of zones,countries,modelling zones,
108,./runs/base-mobility-fullrun-20200527T230910-3...,v5,2020-05-27,14,14,"{'Austria': True, 'Belgium': True, 'Denmark': ...","{'Austria': True, 'Belgium': True, 'Denmark': ...",
109,./runs/base-mobility-fullrun-20200527T230906-3...,v5,2020-05-27,14,30,"{'Austria': True, 'Belgium': True, 'Denmark': ...","{'Austria': True, 'Auvergne-Rhône-Alpes': True...",
110,./runs/base-mobility-fullrun-20200528T223256-3...,v5,2020-05-28,1,16,{'France': True},"{'Auvergne-Rhône-Alpes': True, 'Bourgogne-Fran...",
111,./runs/base-mobility-fullrun-20200528T222424-3...,v5,2020-05-28,14,14,"{'Austria': True, 'Belgium': True, 'Denmark': ...","{'Austria': True, 'Belgium': True, 'Denmark': ...",
112,./runs/base-mobility-fullrun-20200528T222425-3...,v5,2020-05-28,14,30,"{'Austria': True, 'Belgium': True, 'Denmark': ...","{'Austria': True, 'Auvergne-Rhône-Alpes': True...",


## Processing functions

All of them include an overwrite guard in the form of function `safe_write`

```python
    safe_write(file_name, text)
```
to avoid destroying previously written data.


In [40]:
def safe_write(file_name, text):
    if not os.path.exists(file_name):
        with open(file_name, "w", encoding="utf-8") as f:
            print("Writing: " + file_name)
            f.write(text)

### Missing argument list

Copies standard arguments and sets `"max_date"` and `"StanModel"` correctly (for runs <`v5`).

In [39]:
def print_missing_cmd_args(row):
    folder_name = row["folder"].strip()
    file_name = os.path.join(folder_name, os.path.basename(folder_name) + '-inputs-parsed-cmd-arguments.csv')
    
    last_dat = row["last available data"].strip()
    text = f'"","x"\n"DEBUG","FALSE"\n"FULL","TRUE"\n"new_sub_folder","TRUE"\n"max_date","{last_dat}"\n'
    text += '"StanModel","base"\n'
    safe_write(file_name, text)
    

### Missing mobility

Makes an epty csv with mobility columns.

In [41]:
def print_missing_mobility(row):
    folder_name = row["folder"].strip()
    file_name = os.path.join(folder_name, os.path.basename(folder_name) + '-inputs-processed-mobility.csv')
    
    last_dat = row["last available data"].strip()
    text = f'"","date","country","grocery.pharmacy","parks","residential","retail.recreation","transitstations","workplace","schools_universities","self_isolating_if_ill","public_events","lockdown","social_distancing_encouraged"\n'
    text += '"StanModel","base"\n'
    
    safe_write(file_name, text)

## Process all rows

Thanks to the `safe_write` function the following lines have an effect only on first run.

In [43]:
for i, row in run_manifest.iterrows():
    print_missing_cmd_args(row)
    print_missing_mobility(row)